<a href="https://colab.research.google.com/github/SahilKhan101/IPL-Data-Analysis/blob/main/WiDS_IPL_ML_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,SGDRegressor,ElasticNet
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.validation import check_array 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from sklearn.svm import LinearSVR, SVC, SVR
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor

### Steps to be Executed

In [ ]:
#1st Step

"""
No NAN values
All features should be numerical (can use label encoding or pd.dummies to convert categorical data into numerical data)
use MinMaxScaler or StandardScaler (to bring all features to the same scale)
"""
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [ ]:
#2nd Step

"""
Perform correlation analysis (to find similar features, then remove similar features)
"""
# prediction_df=matches[["team1","team2","team1_toss_win","team1_bat","team1_win","venue"]]
# #finding the higly correlated features
# correlated_features = set()
# correlation_matrix = prediction_df.drop('team1_win', axis=1).corr()
# for i in range(len(correlation_matrix.columns)):
#     for j in range(i):
#         if abs(correlation_matrix.iloc[i, j]) > 0.9:
#             column = correlation_matrix.columns[i]
#             correlated_features.add(column)
# prediction_df.drop(columns=correlated_features)

"""
make at least 2 new features which would be useful to predict your target
https://colab.research.google.com/drive/1dnT8wZo3-SNwUbvuwjGzBz4amuUteAqg?usp=sharing#scrollTo=qv98PJ5UMv9I
"""

In [ ]:
#3rd Step

"""
Use at least three different ML models for each task
For classification tasks: RF, Logit, DTC, XGB, SVM (https://towardsdatascience.com/predicting-ipl-match-winner-fc9e89f583ce)
For Regression tasks: Linear Reg, RF, SVR
Compare the metrics for all the models in a pandas dataframe
"""

In [ ]:
#4th Step

# def custom_accuracy2(y_test,y_pred,thresold):
#     right = 0
#     l = len(y_pred)
#     for i in range(0,l):
#         if(abs(y_pred[i]-y_test[i]) <= thresold):
#             right += 1
#     return ((right/l)*100)

# Classification Task

## Data Cleaning 

In [ ]:
matches_data = pd.read_csv('matches.csv')
null_city_list = list(matches_data['city'].isnull())

#imputing the values in column city based on venue
matches = matches_data.copy()
conditions = [matches["venue"] == "Rajiv Gandhi International Stadium, Uppal",matches["venue"] == "Maharashtra Cricket Association Stadium",
              matches["venue"] == "Saurashtra Cricket Association Stadium", matches["venue"] == "Holkar Cricket Stadium",
              matches["venue"] == "M Chinnaswamy Stadium",matches["venue"] == "Wankhede Stadium",
              matches["venue"] == "Eden Gardens",matches["venue"] == "Feroz Shah Kotla",
              matches["venue"] == "Punjab Cricket Association IS Bindra Stadium, Mohali",matches["venue"] == "Green Park",
              matches["venue"] == "Punjab Cricket Association Stadium, Mohali",matches["venue"] == "Dr DY Patil Sports Academy",
              matches["venue"] == "Sawai Mansingh Stadium", matches["venue"] == "MA Chidambaram Stadium, Chepauk", 
              matches["venue"] == "Newlands", matches["venue"] == "St George's Park" , 
              matches["venue"] == "Kingsmead", matches["venue"] == "SuperSport Park",
              matches["venue"] == "Buffalo Park", matches["venue"] == "New Wanderers Stadium",
              matches["venue"] == "De Beers Diamond Oval", matches["venue"] == "OUTsurance Oval", 
              matches["venue"] == "Brabourne Stadium",matches["venue"] == "Sardar Patel Stadium", 
              matches["venue"] == "Barabati Stadium", matches["venue"] == "Vidarbha Cricket Association Stadium, Jamtha",
              matches["venue"] == "Himachal Pradesh Cricket Association Stadium",matches["venue"] == "Nehru Stadium",
              matches["venue"] == "Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium",matches["venue"] == "Subrata Roy Sahara Stadium",
              matches["venue"] == "Shaheed Veer Narayan Singh International Stadium",matches["venue"] == "JSCA International Stadium Complex",
              matches["venue"] == "Sheikh Zayed Stadium",matches["venue"] == "Sharjah Cricket Stadium",
              matches["venue"] == "Dubai International Cricket Stadium",matches["venue"] == "M. A. Chidambaram Stadium",
              matches["venue"] == "Feroz Shah Kotla Ground",matches["venue"] == "M. Chinnaswamy Stadium",
              matches["venue"] == "Rajiv Gandhi Intl. Cricket Stadium" ,matches["venue"] == "IS Bindra Stadium",matches["venue"] == "ACA-VDCA Stadium"]
values = ['Hyderabad', 'Mumbai', 'Rajkot',"Indore","Bengaluru","Mumbai","Kolkata","Delhi","Mohali","Kanpur","Mohali","Pune","Jaipur","Chennai","Cape Town","Port Elizabeth","Durban",
          "Centurion",'Eastern Cape','Johannesburg','Northern Cape','Bloemfontein','Mumbai','Ahmedabad','Cuttack','Jamtha','Dharamshala','Chennai','Visakhapatnam','Pune','Raipur','Ranchi',
          'Abu Dhabi','Sharjah','Dubai','Chennai','Delhi','Bengaluru','Hyderabad','Mohali','Visakhapatnam']
matches['city'] = np.where(matches['city'].isnull(),
                              np.select(conditions, values),
                              matches['city'])
matches_data = matches.copy()

#Clearing Ambiguity regarding same team name in different fasion
matches_data['team1'] = matches_data['team1'].replace("Rising Pune Supergiant","Rising Pune Supergiants")
matches_data['team2'] = matches_data['team2'].replace("Rising Pune Supergiant","Rising Pune Supergiants")
matches_data['winner'] = matches_data['winner'].replace("Rising Pune Supergiant","Rising Pune Supergiants")
matches_data['toss_winner'] = matches_data['toss_winner'].replace("Rising Pune Supergiant","Rising Pune Supergiants")

#Removing Null values
matches_data = matches_data[matches_data['winner'].notna()]

#Encoding
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
matches_data['team1'] = encoder.fit_transform(matches_data['team1'])
matches_data['team2'] = encoder.fit_transform(matches_data['team2'])
matches_data['winner'] = encoder.fit_transform(matches_data['winner'].astype(str))
matches_data['toss_winner'] = encoder.fit_transform(matches_data['toss_winner'])
matches_data['venue'] = encoder.fit_transform(matches_data['venue'])


#Feature Engineering
matches_data.loc[matches_data['winner']==matches_data['team1'], 'team1_win']=1    #"team1_win"
matches_data.loc[matches_data['winner']!=matches_data['team1'], 'team1_win']=0

matches_data.loc[matches_data["toss_winner"]==matches_data["team1"],"team1_toss_win"]=1   #"team1_toss_win" 
matches_data.loc[matches_data["toss_winner"]!=matches_data["team1"],"team1_toss_win"]=0

matches_data["team1_bat"]=0     #"team1_bat"
matches_data.loc[((matches_data["team1_toss_win"]==1) & (matches_data["toss_decision"]=="bat")) | ((matches_data["team1_toss_win"]==0) & (matches_data["toss_decision"]=="field")) ,"team1_bat"]=1

#Removing unnecessary columns
sign_matches = matches_data.drop(columns=['umpire1', 'umpire2', 'umpire3'])

#Copying Dataframe to matches variable
matches = sign_matches.copy()

## Feature selection

In [ ]:
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,team1_win,team1_toss_win,team1_bat
0,1,2017,Hyderabad,2017-04-05,13,12,12,field,normal,0,13,35,0,Yuvraj Singh,28,1.0,0.0,1
1,2,2017,Pune,2017-04-06,8,11,11,field,normal,0,11,0,7,SPD Smith,21,0.0,0.0,1
2,3,2017,Rajkot,2017-04-07,4,7,7,field,normal,0,7,0,10,CA Lynn,31,0.0,0.0,1
3,4,2017,Indore,2017-04-08,11,5,5,field,normal,0,5,0,6,GJ Maxwell,13,0.0,0.0,1
4,5,2017,Bangalore,2017-04-08,12,3,12,bat,normal,0,12,15,0,KM Jadhav,17,1.0,1.0,1


In [ ]:
matches = matches[['id', 'season', 'team1', 'team2', 'winner', 'team1_win', 'team1_toss_win', 'team1_bat', 'venue', 'win_by_runs', 'win_by_wickets']]

In [ ]:
matches.head()

,id,season,team1,team2,winner,team1_win,team1_toss_win,team1_bat,venue,win_by_runs,win_by_wickets
0,1,2017,13,12,13,1.0,0.0,1,28,35,0
1,2,2017,8,11,11,0.0,0.0,1,21,0,7
2,3,2017,4,7,7,0.0,0.0,1,31,0,10
3,4,2017,11,5,5,0.0,0.0,1,13,0,6
4,5,2017,12,3,12,1.0,1.0,1,17,15,0


In [ ]:
matches['team1_bat'].unique()

array([1], dtype=int64)

In [ ]:
#Winning stats of teams bat/field first by venues
# venue_toss_decision_result=matches[["team1_toss_win","team1_bat","team1_win","venue"]]
# venue_toss_decision_result["decision"]=np.where((venue_toss_decision_result.team1_toss_win == venue_toss_decision_result.team1_win) & (venue_toss_decision_result.team1_bat == 0),"field_won","bat_won")
# venue_result=venue_toss_decision_result.groupby(["venue"]).decision.value_counts().reset_index(name="count")
# #Visualization
# sns.set(rc={'figure.figsize':(20,10)})
# plot = sns.barplot(x="venue", y="count", hue="decision", data=venue_result)
# plot.set_title('Teams bat/field first results on venues')
# plot.set_xticklabels(venue_result['venue'].unique(),rotation=90)
# plt.show()

In [ ]:
#Correlation Analysis (to find similar features, then remove similar features)

prediction_df=matches[["team1","team2","team1_toss_win","team1_bat","team1_win","venue"]]
correlated_features = set()
correlation_matrix = prediction_df.drop('team1_win', axis=1).corr()
print(correlation_matrix)
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            column = correlation_matrix.columns[i]
            correlated_features.add(column)
prediction_df.drop(columns=correlated_features)

                   team1     team2  team1_toss_win  team1_bat     venue
team1           1.000000 -0.108116       -0.116832        NaN  0.108240
team2          -0.108116  1.000000       -0.008782        NaN  0.084187
team1_toss_win -0.116832 -0.008782        1.000000        NaN  0.050049
team1_bat            NaN       NaN             NaN        NaN       NaN
venue           0.108240  0.084187        0.050049        NaN  1.000000


,team1,team2,team1_toss_win,team1_bat,team1_win,venue
0,13,12,0.0,1,1.0,28
1,8,11,0.0,1,0.0,21
2,4,7,0.0,1,0.0,31
3,11,5,0.0,1,0.0,13
4,12,3,1.0,1,1.0,17
...,...,...,...,...,...,...
751,7,8,0.0,1,0.0,40
752,0,8,1.0,1,0.0,18
753,13,2,0.0,1,0.0,0
754,2,0,0.0,1,0.0,0


## Train-Test split

In [ ]:
X = prediction_df.drop('team1_win', axis=1)
y = prediction_df['team1_win'].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#StandardScaler (to bring all features to the same scale)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## ML Models

### Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(X_test, y_test)))

[[76  0]
 [75  0]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        76
           1       0.00      0.00      0.00        75

    accuracy                           0.50       151
   macro avg       0.25      0.50      0.33       151
weighted avg       0.25      0.50      0.34       151

Accuracy of logistic regression classifier on test set: 0.5033


### SVM

In [ ]:
svm = SVC()
svm.fit(X_train,y_train)
svm.score(X_test,y_test)
y_pred = svm.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print('Accuracy of SVM classifier on test set: {:.4f}'.format(svm.score(X_test, y_test)))

[[67  9]
 [63 12]]
              precision    recall  f1-score   support

           0       0.52      0.88      0.65        76
           1       0.57      0.16      0.25        75

    accuracy                           0.52       151
   macro avg       0.54      0.52      0.45       151
weighted avg       0.54      0.52      0.45       151

Accuracy of SVM classifier on test set: 0.5232


### Decision Tree Classifier

In [ ]:
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)
dtree.score(X_test,y_test)
y_pred = dtree.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print('Accuracy of decision tree classifier on test set: {:.4f}'.format(dtree.score(X_test, y_test)))

[[51 25]
 [44 31]]
              precision    recall  f1-score   support

           0       0.54      0.67      0.60        76
           1       0.55      0.41      0.47        75

    accuracy                           0.54       151
   macro avg       0.55      0.54      0.53       151
weighted avg       0.55      0.54      0.54       151

Accuracy of decision tree classifier on test set: 0.5430


### Random Forest Classifier

In [ ]:
randomForest= RandomForestClassifier(n_estimators=100)
randomForest.fit(X_train,y_train)
randomForest.score(X_test,y_test)
y_pred = randomForest.predict(X_test)
print("Confusion matrix\n",confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print('Accuracy of random forest classifier on test set: {:.4f}'.format(randomForest.score(X_test, y_test)))

Confusion matrix
 [[52 24]
 [45 30]]
              precision    recall  f1-score   support

           0       0.54      0.68      0.60        76
           1       0.56      0.40      0.47        75

    accuracy                           0.54       151
   macro avg       0.55      0.54      0.53       151
weighted avg       0.55      0.54      0.53       151

Accuracy of random forest classifier on test set: 0.5430


### Conclusion

In [ ]:
df = [['Logistic Regression', logreg.score(X_test, y_test)], 
      ['SVM', svm.score(X_test, y_test)], 
      ['Decision Tree Classifier', dtree.score(X_test, y_test)], 
      ['Random Forest Classifier', randomForest.score(X_test, y_test)]]

In [ ]:
metric_comparison = pd.DataFrame(df, columns=['model', 'f1_score'])

In [ ]:
metric_comparison

,model,f1_score
0,Logistic Regression,0.503311
1,SVM,0.523179
2,Decision Tree Classifier,0.543046
3,Random Forest Classifier,0.543046


So accuracy of  **Decisoin Tree** and **Random Forest** are nearly equal, so compairing as per confusion matrix, best suitable model for this task is ***Random Forest Classifier***

# Regression Task

## Data Collection

In [ ]:
balls = pd.read_csv('IPL Ball-by-Ball 2008-2020.csv')
print(balls.shape)
balls.head()

(193468, 18)


,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [ ]:
matches = pd.read_csv('IPL Matches 2008-2020.csv')
print(matches.shape)
matches.head()

(816, 17)


,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [ ]:
ipl_data = pd.merge(left=matches, right=balls, on='id')
print(ipl_data.shape)
ipl_data.head()

(193468, 34)


,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


## Helper Functions

In [ ]:
def make_count_df(sign_matches, target_col, col1, col2):
    team_win_ser = sign_matches[target_col].value_counts()
    team_win_df = pd.DataFrame(columns=[col1, col2])
    for items in team_win_ser.iteritems():
        temp_df = pd.DataFrame({
            col1:[items[0]],
            col2:[items[1]]
        })
        team_win_df = team_win_df.append(temp_df, ignore_index=True) 

    return team_win_df

## Data Preprocessing

In [ ]:
ipl_data['year'] = pd.DatetimeIndex(pd.to_datetime(ipl_data['date'])).year

In [ ]:
make_count_df(ipl_data, 'team1', 'team', 'No. of appearance in df')

,team,No. of appearance in df
0,Royal Challengers Bangalore,25238
1,Mumbai Indians,23335
2,Chennai Super Kings,22740
3,Kolkata Knight Riders,22318
4,Kings XI Punjab,21805
5,Delhi Daredevils,19286
6,Rajasthan Royals,16679
7,Sunrisers Hyderabad,13879
8,Deccan Chargers,9448
9,Pune Warriors,5483


### Modifying Data based on historical information
1) On December 5th 2018, the team of **Delhi Daredevils** got its name changed to **Delhi Capitals** <br>
2) **Pune Warriors India** and **Rising Pune Supergiant** were the two teams that represented the city in **different years** 
<br>
3) The owners changed the team name **Rising Pune Supergiants** to **Rising Pune Supergiant** on 26 March 2017

In [ ]:
ipl_data = ipl_data.replace(['Rising Pune Supergiants','Rising Pune Supergiant', 'Delhi Daredevils'],['Pune Warriors','Pune Warriors', 'Delhi Capitals'])

In [ ]:
make_count_df(ipl_data, 'team1', 'team', 'No. of appearance in df')

,team,No. of appearance in df
0,Royal Challengers Bangalore,25238
1,Delhi Capitals,23915
2,Mumbai Indians,23335
3,Chennai Super Kings,22740
4,Kolkata Knight Riders,22318
5,Kings XI Punjab,21805
6,Rajasthan Royals,16679
7,Sunrisers Hyderabad,13879
8,Deccan Chargers,9448
9,Pune Warriors,8777


### Removing Null values

In [ ]:
ipl_data.isnull().sum()

id                       0
city                  3139
date                     0
player_of_match        372
venue                    0
neutral_venue            0
team1                    0
team2                    0
toss_winner              0
toss_decision            0
winner                 372
result                 372
result_margin         3597
eliminator             372
method              190260
umpire1                  0
umpire2                  0
inning                   0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
batsman_runs             0
extra_runs               0
total_runs               0
non_boundary             0
is_wicket                0
dismissal_kind      183973
player_dismissed    183973
fielder             186684
extras_type         183235
batting_team             0
bowling_team           191
year                     0
dtype: int64

In [ ]:
drop_columns = ['city', 'result_margin', 'method', 'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type', 'bowling_team']
ipl_data.drop(drop_columns,axis=1, inplace=True)

In [ ]:
ipl_data = ipl_data[ipl_data['winner'].notna()]

In [ ]:
ipl_data.isnull().sum()

id                 0
date               0
player_of_match    0
venue              0
neutral_venue      0
team1              0
team2              0
toss_winner        0
toss_decision      0
winner             0
result             0
eliminator         0
umpire1            0
umpire2            0
inning             0
over               0
ball               0
batsman            0
non_striker        0
bowler             0
batsman_runs       0
extra_runs         0
total_runs         0
non_boundary       0
is_wicket          0
batting_team       0
year               0
dtype: int64

### Feature Engineering

In [ ]:
ipl_data['inning'] = ipl_data['id'].astype(str)+(ipl_data['batting_team']).astype(str)
ipl_data.head()

,id,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,eliminator,umpire1,umpire2,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,batting_team,year
0,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,335982Kolkata Knight Riders,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,Kolkata Knight Riders,2008
1,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,335982Kolkata Knight Riders,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,Kolkata Knight Riders,2008
2,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,335982Kolkata Knight Riders,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,Kolkata Knight Riders,2008
3,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,335982Kolkata Knight Riders,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,Kolkata Knight Riders,2008
4,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,335982Kolkata Knight Riders,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,Kolkata Knight Riders,2008


In [ ]:
unique_inning_list = ipl_data['inning'].unique()

In [ ]:
ipl_data.loc[(ipl_data['over'] == 10) & (ipl_data['ball'] == 6) & (ipl_data['inning'] == unique_inning_list[0])]

,id,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,eliminator,umpire1,umpire2,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,batting_team,year
25,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,335982Kolkata Knight Riders,10,6,RT Ponting,BB McCullum,JH Kallis,0,4,4,0,0,Kolkata Knight Riders,2008


In [ ]:
df = ipl_data.copy()
df = df.groupby(by=['inning']).sum()['total_runs'].reset_index()
df.head()

,inning,total_runs
0,1082591Royal Challengers Bangalore,172
1,1082591Sunrisers Hyderabad,207
2,1082592Mumbai Indians,184
3,1082592Pune Warriors,187
4,1082593Gujarat Lions,183


In [ ]:
df2 = pd.merge(left=df, right=ipl_data, on='inning', how='right')
print(df2.shape, ipl_data.shape)
df2.head()

(193096, 28) (193096, 27)


,inning,total_runs_x,id,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,eliminator,umpire1,umpire2,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs_y,non_boundary,is_wicket,batting_team,year
0,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,Kolkata Knight Riders,2008
1,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,Kolkata Knight Riders,2008
2,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,Kolkata Knight Riders,2008
3,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,Kolkata Knight Riders,2008
4,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,Kolkata Knight Riders,2008


In [ ]:
df3 = pd.DataFrame()
for inn in unique_inning_list:
    one_inning = df2.loc[df2['inning']==inn]
    one_inning = one_inning.sort_values(['over', 'ball'], ascending=[True, True])
    one_inning['cumsum_runs'] = one_inning['total_runs_y'].cumsum()
    one_inning['cumsum_wickets'] = one_inning['is_wicket'].cumsum()
    df3 = pd.concat([df3, one_inning])
print(df2.shape, df3.shape)

(193096, 28) (193096, 30)


In [ ]:
df3.head()

,inning,total_runs_x,id,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,eliminator,umpire1,umpire2,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs_y,non_boundary,is_wicket,batting_team,year,cumsum_runs,cumsum_wickets
79,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0,1,SC Ganguly,BB McCullum,P Kumar,0,1,1,0,0,Kolkata Knight Riders,2008,1,0
80,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0,2,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,Kolkata Knight Riders,2008,1,0
81,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0,3,BB McCullum,SC Ganguly,P Kumar,0,1,1,0,0,Kolkata Knight Riders,2008,2,0
82,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0,4,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,Kolkata Knight Riders,2008,2,0
83,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0,5,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,Kolkata Knight Riders,2008,2,0


In [ ]:
df3['over'] = df3['over']+ (df3['ball'])/10
df3 = df3.drop(['ball'], axis=1)

In [ ]:
ipl_data = df3.copy()

In [ ]:
ipl_data.head()

,inning,total_runs_x,id,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,eliminator,umpire1,umpire2,over,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs_y,non_boundary,is_wicket,batting_team,year,cumsum_runs,cumsum_wickets
79,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0.1,SC Ganguly,BB McCullum,P Kumar,0,1,1,0,0,Kolkata Knight Riders,2008,1,0
80,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0.2,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,Kolkata Knight Riders,2008,1,0
81,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0.3,BB McCullum,SC Ganguly,P Kumar,0,1,1,0,0,Kolkata Knight Riders,2008,2,0
82,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0.4,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,Kolkata Knight Riders,2008,2,0
83,335982Kolkata Knight Riders,222,335982,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,N,Asad Rauf,RE Koertzen,0.5,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,Kolkata Knight Riders,2008,2,0


### Encoding

In [ ]:
matches_data = ipl_data.copy()
encoder = preprocessing.LabelEncoder()
matches_data['team1'] = encoder.fit_transform(matches_data['team1'])
matches_data['team2'] = encoder.fit_transform(matches_data['team2'])
matches_data['winner'] = encoder.fit_transform(matches_data['winner'].astype(str))
matches_data['toss_winner'] = encoder.fit_transform(matches_data['toss_winner'])
matches_data['venue'] = encoder.fit_transform(matches_data['venue'])
matches_data['batting_team'] = encoder.fit_transform(matches_data['batting_team'])
matches_data['player_of_match'] = encoder.fit_transform(matches_data['player_of_match'])

matches_data.loc[matches_data['winner']==matches_data['team1'], 'team1_win']=1    #"team1_win"
matches_data.loc[matches_data['winner']!=matches_data['team1'], 'team1_win']=0
matches_data.loc[matches_data["toss_winner"]==matches_data["team1"],"team1_toss_win"]=1   #"team1_toss_win" 
matches_data.loc[matches_data["toss_winner"]!=matches_data["team1"],"team1_toss_win"]=0
matches_data["team1_bat"]=0     #"team1_bat"
matches_data.loc[((matches_data["team1_toss_win"]==1) & (matches_data["toss_decision"]=="bat")) | ((matches_data["team1_toss_win"]==0) & (matches_data["toss_decision"]=="field")) ,"team1_bat"]=1

ipl_data = matches_data.copy()
matches_data.head()

,inning,total_runs_x,id,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,eliminator,umpire1,umpire2,over,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs_y,non_boundary,is_wicket,batting_team,year,cumsum_runs,cumsum_wickets,team1_win,team1_toss_win,team1_bat
79,335982Kolkata Knight Riders,222,335982,2008-04-18,30,14,0,10,6,10,field,6,runs,N,Asad Rauf,RE Koertzen,0.1,SC Ganguly,BB McCullum,P Kumar,0,1,1,0,0,6,2008,1,0,0.0,1.0,0
80,335982Kolkata Knight Riders,222,335982,2008-04-18,30,14,0,10,6,10,field,6,runs,N,Asad Rauf,RE Koertzen,0.2,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,6,2008,1,0,0.0,1.0,0
81,335982Kolkata Knight Riders,222,335982,2008-04-18,30,14,0,10,6,10,field,6,runs,N,Asad Rauf,RE Koertzen,0.3,BB McCullum,SC Ganguly,P Kumar,0,1,1,0,0,6,2008,2,0,0.0,1.0,0
82,335982Kolkata Knight Riders,222,335982,2008-04-18,30,14,0,10,6,10,field,6,runs,N,Asad Rauf,RE Koertzen,0.4,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,6,2008,2,0,0.0,1.0,0
83,335982Kolkata Knight Riders,222,335982,2008-04-18,30,14,0,10,6,10,field,6,runs,N,Asad Rauf,RE Koertzen,0.5,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,6,2008,2,0,0.0,1.0,0


### Removing Unnecessary Features

In [ ]:
useful_columns = ['inning', 'id', 'player_of_match', 'player_of_match', 'venue', 'team1', 'team2', 'over', 'batsman_runs', 'year', 'cumsum_runs', 'cumsum_wickets', 'team1_win', 'team1_toss_win', 'team1_bat', 'total_runs_x']
ipl_data = ipl_data[useful_columns]

In [ ]:
ipl_data

,inning,id,player_of_match,player_of_match,venue,team1,team2,over,batsman_runs,year,cumsum_runs,cumsum_wickets,team1_win,team1_toss_win,team1_bat,total_runs_x
79,335982Kolkata Knight Riders,335982,30,30,14,10,6,0.1,0,2008,1,0,0.0,1.0,0,222
80,335982Kolkata Knight Riders,335982,30,30,14,10,6,0.2,0,2008,1,0,0.0,1.0,0,222
81,335982Kolkata Knight Riders,335982,30,30,14,10,6,0.3,0,2008,2,0,0.0,1.0,0,222
82,335982Kolkata Knight Riders,335982,30,30,14,10,6,0.4,0,2008,2,0,0.0,1.0,0,222
83,335982Kolkata Knight Riders,335982,30,30,14,10,6,0.5,0,2008,2,0,0.0,1.0,0,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193091,1237181Mumbai Indians,1237181,217,217,6,2,7,17.6,1,2020,154,4,0.0,1.0,1,157
193092,1237181Mumbai Indians,1237181,217,217,6,2,7,18.1,1,2020,155,4,0.0,1.0,1,157
193093,1237181Mumbai Indians,1237181,217,217,6,2,7,18.2,1,2020,156,4,0.0,1.0,1,157
193094,1237181Mumbai Indians,1237181,217,217,6,2,7,18.3,0,2020,156,5,0.0,1.0,1,157


### Train Test Split

In [ ]:
ipl_data = pd.read_csv('ipl_data.csv', index_col=0)

In [ ]:
pred_features = ['cumsum_runs', 'cumsum_wickets', 'over', 'venue', 'team1', 'team2', 'team1_bat']
#venue,team1,team2,over,batsman_runs,team1_toss_win,team1_bat
X = ipl_data[pred_features]
y = ipl_data['total_runs_x'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Custom Accuracy function

In [ ]:
def custom_accuracy(y_test,y_pred,thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right += 1
    return ((right/l)*100)

## Models

### Linear Regression

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
lm_metric = [mean_squared_error(y_test, pred), custom_accuracy(y_test, pred, 10)]
print("Custom accuracy: {:.2f} %".format(lm_metric[1]))

Custom accuracy: 41.73 %


### Random Forest

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
rf_metric = [mean_squared_error(y_test, pred), custom_accuracy(y_test, pred, 10)]
print("Custom accuracy: {:.2f} %".format(rf_metric[1]))

Custom accuracy: 76.19 %


### Linear SVR

In [ ]:
svr = LinearSVR()
svr.fit(X_train, y_train)
pred = svr.predict(X_test)
svr_metric = [mean_squared_error(y_test, pred), custom_accuracy(y_test, pred, 10)]
print("Custom accuracy: {:.2f} %".format(svr_metric[1]))

Custom accuracy: 42.08 %


## Conclusion

In [ ]:
df = [ ['Linear Regression', lm_metric[0], lm_metric[1]], ['Random Forest', rf_metric[0], rf_metric[1]], ['Linear SVR', svr_metric[0], svr_metric[1]] ]
pd.DataFrame(df, columns=['models','mean squared error', 'custom accuracy'])

,models,mean squared error,custom accuracy
0,Linear Regression,522.667463,41.727084
1,Random Forest,156.065917,76.185914
2,Linear SVR,525.211956,42.076644


**Random Forest** give the largest accuracy among these three models